In [1]:
# To fix the encoding error with python2
import sys
reload(sys)
sys.setdefaultencoding('utf8')

import os
import random
from time import time
import string
import re
from nltk.corpus.reader.bracket_parse import BracketParseCorpusReader

CORPUS = os.path.join("corpus", "sequoia-corpus+fct.mrg_strict")
TB_TRAIN = os.path.join("corpus", "sequoia_train.tb")
TB_DEV = os.path.join("corpus", "sequoia_dev.tb")
TB_TEST = os.path.join("corpus", "sequoia_test.tb")
TXT_DEV = os.path.join("corpus", "sequoia_dev.txt")
TXT_TEST = os.path.join("corpus", "sequoia_test.txt")

t0 = time()
print (">>> Splitting corpus into train/dev/test sets...")

nt_funcl_re = re.compile(r"(?<=\()[A-Za-z_+^\-]+\-[^ ]+")
def remove_funcl(m):
    return m.group().split('-')[0]

f_in = open(CORPUS, 'r')
data = f_in.read().splitlines()
for i in range(len(data)):
    data[i] = nt_funcl_re.sub(lambda x: remove_funcl(x), data[i])
f_in.close()

s_total = len(data)
p_train = 0.8
p_dev = 0.1
p_test = 0.1

random.seed(39)
random.shuffle(data)

corpus_train = data[:int(s_total * p_train)]
f_train = open(TB_TRAIN, 'w')
for s in corpus_train:
    f_train.write("%s\n" % s)
f_train.close()

corpus_dev = data[int(s_total * p_train) : int(s_total * (p_train + p_dev))]
f_dev = open(TB_DEV, 'w')
for s in corpus_dev:
    f_dev.write("%s\n" % s)
f_dev.close()

corpus_test = data[int(s_total * (p_train + p_dev)):]
f_test = open(TB_TEST, 'w')
for s in corpus_test:
    f_test.write("%s\n" % s)
f_test.close()

corpus_root = r"./corpus/"

dev_file_pattern = r".*_dev\.tb"
ptb_dev = BracketParseCorpusReader(corpus_root, dev_file_pattern)
trees = ptb_dev.parsed_sents()
f_out = open(TXT_DEV, 'w')
for tree in trees:
    f_out.write("%s\n" % (' '.join(tree.leaves())))
f_out.close()

test_file_pattern = r".*_test\.tb"
ptb_test = BracketParseCorpusReader(corpus_root, test_file_pattern)
trees = ptb_test.parsed_sents()
f_out = open(TXT_TEST, 'w')
for tree in trees:
    f_out.write("%s\n" % (' '.join(tree.leaves())))
f_out.close()

print ("Corpus size: %d" % s_total)
print ("Train set size: %d" % len(corpus_train))
print ("Dev set size: %d" % len(corpus_dev))
print ("Test set size: %d" % len(corpus_test))

print (">>> Corpus split done in %0.3fs.\n" % (time() - t0))